## Covid-19 Basic Model

In this challenge, you will be predicting the cumulative number of confirmed COVID19 cases in various locations across the world, as well as the number of resulting fatalities, for future dates.

당신은 미래의 사망자 수뿐만 아니라 전 세계의 여러 위치에서 확인 된 COVID19 사례의 누적 수를
예측할 수 있다.

Files
train.csv - the training data


test.csv - the dates to predict


submission.csv

## Import / Load and Inspect data

In [1]:
import numpy as np
import pandas as pd

df = pd.read_csv("./train.csv")
# df.shape (행, 열)
df.shape

(23562, 6)

In [2]:
df.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,NaN,Afghanistan,2020-01-22,0.0,0.0
1,2,NaN,Afghanistan,2020-01-23,0.0,0.0
2,3,NaN,Afghanistan,2020-01-24,0.0,0.0
3,4,NaN,Afghanistan,2020-01-25,0.0,0.0
4,5,NaN,Afghanistan,2020-01-26,0.0,0.0


In [3]:
type(df["Date"].values)

numpy.ndarray

In [4]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 23562 entries, 0 to 23561
Data columns (total 6 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   Id              23562 non-null  int64  
 1   Province_State  10010 non-null  object 
 2   Country_Region  23562 non-null  object 
 3   Date            23562 non-null  object 
 4   ConfirmedCases  23562 non-null  float64
 5   Fatalities      23562 non-null  float64
dtypes: float64(2), int64(1), object(3)
memory usage: 1.1+ MB


In [5]:
# 확인해야 할 행 그룹
# Province_State : 지역, 주 ( 경기도 )
# Country_Region : 나라
loc_group = ["Province_State", "Country_Region"]

# 결측값(NaN)을 none으로 채우는 함수
# datetime64[ms] : 밀리 초 단위 주기
# df.astype : pandas 객체를 지정된 객체로 type변환
# fillna : NaN을 특정 값으로 대체한다.
def preprocess(df):
    df["Date"] = df["Date"].astype("datetime64[ms]")
    for col in loc_group:
        df[col].fillna("none", inplace=True) # NaN을 "none"으로 대체
    return df                               # inplace = 원본 변환 유무

df = preprocess(df)   # 함수 호출
sub_df = preprocess(pd.read_csv("./test.csv")) 
# test.csv의 NaN값 none 대체
df.head()        # 상위 5개만 data 를 보여준다

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities
0,1,none,Afghanistan,2020-01-22,0.0,0.0
1,2,none,Afghanistan,2020-01-23,0.0,0.0
2,3,none,Afghanistan,2020-01-24,0.0,0.0
3,4,none,Afghanistan,2020-01-25,0.0,0.0
4,5,none,Afghanistan,2020-01-26,0.0,0.0


In [6]:
# 첫 covid19 발생 시작일과 가장 마지막 날 출력
df["Date"].min(), df["Date"].max()

(Timestamp('2020-01-22 00:00:00'), Timestamp('2020-04-07 00:00:00'))

In [7]:
# Confirmed Cases : 코로나 확인된 case (ex. 우리나라 10384 case)
# Fatalities : 사망자
TARGETS = ["ConfirmedCases", "Fatalities"]
# 코로나 확진자와 사망자를 TARGETS으로 삼아
# np.log1p : 입력 값에 대해 자연로그 log(1 + x) 값을 반환
# np.log1p를 사용하는 이유 ? 왜도를 없애주고/ 더 높은 정확도 제공/ NaN값에 대해서 알아서 오류플래그 설정
for col in TARGETS:
    df[col] = np.log1p(df[col])

In [8]:
df.groupby(loc_group)[col].shift()

0             NaN
1        0.000000
2        0.000000
3        0.000000
4        0.000000
           ...   
23557    0.693147
23558    0.693147
23559    0.693147
23560    0.693147
23561    0.693147
Name: Fatalities, Length: 23562, dtype: float64

In [9]:
# loc_group = ["Province_State", "Country_Region"]
# .format : 괄호를 사용하는 포맷팅 방법
# {}.format(col) : 괄호 안에 col 삽입

for col in TARGETS:    # TARGETS = ["ConfirmedCases", "Fatalities"]
    df["prev_{}".format(col)] = df.groupby(loc_group)[col].shift()
# df.groupby( 기준이 될 열 형식 ) 
# loc_group의 열을 데이터 그룹화
# dfgroupby.shift(periods=1, freq=None, axis=0) : 각각의 그룹을 이동
# axis = 0 : 인덱스 이동 / axis = 1 : 행 이동

# 즉 df의 열을 두개 추가 : prev_ConfirmedCases와 prev_Fatalities
# 각각의 그룹을 1 인덱스 이동

In [10]:
df[df["Date"] > df["Date"].min()]

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,prev_ConfirmedCases,prev_Fatalities
1,2,none,Afghanistan,2020-01-23,0.000000,0.000000,0.000000,0.000000
2,3,none,Afghanistan,2020-01-24,0.000000,0.000000,0.000000,0.000000
3,4,none,Afghanistan,2020-01-25,0.000000,0.000000,0.000000,0.000000
4,5,none,Afghanistan,2020-01-26,0.000000,0.000000,0.000000,0.000000
5,6,none,Afghanistan,2020-01-27,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
23557,32708,none,Zimbabwe,2020-04-03,2.302585,0.693147,2.302585,0.693147
23558,32709,none,Zimbabwe,2020-04-04,2.302585,0.693147,2.302585,0.693147
23559,32710,none,Zimbabwe,2020-04-05,2.302585,0.693147,2.302585,0.693147
23560,32711,none,Zimbabwe,2020-04-06,2.397895,0.693147,2.302585,0.693147


In [11]:
# 최초 코로나 발생일인 2020-01-24 00:00:00을 제외한 이후에 발생한 날짜들에 대한
# 데이터들을 저장한다.
# df.copy() : 인덱스와 데이터를 복사 (deep = True), default : 사본
df = df[df["Date"] > df["Date"].min()].copy()
df.head()
# 인덱스가 변화된 것 확인

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,prev_ConfirmedCases,prev_Fatalities
1,2,none,Afghanistan,2020-01-23,0.0,0.0,0.0,0.0
2,3,none,Afghanistan,2020-01-24,0.0,0.0,0.0,0.0
3,4,none,Afghanistan,2020-01-25,0.0,0.0,0.0,0.0
4,5,none,Afghanistan,2020-01-26,0.0,0.0,0.0,0.0
5,6,none,Afghanistan,2020-01-27,0.0,0.0,0.0,0.0


*아프가니스탄만 있는걸까?*
- 여러나라가 있는데 아프가니스탄이 첫번째 정렬

In [12]:
TEST_FIRST = sub_df["Date"].min
TEST_FIRST

<bound method Series.min of 0       2020-03-26
1       2020-03-27
2       2020-03-28
3       2020-03-29
4       2020-03-30
           ...    
13153   2020-05-03
13154   2020-05-04
13155   2020-05-05
13156   2020-05-06
13157   2020-05-07
Name: Date, Length: 13158, dtype: datetime64[ns]>

In [13]:
# datetime 모듈은 날짜와 시간을 조작하는 클래스를 제공
from datetime import timedelta

TEST_DAYS = 7
# timedelta 클래스 : 두 날짜 혹은 시간 사이의 기간
TRAIN_LAST =  - timedelta(days=TEST_DAYS)
# 현재로 부터 7일 전으로(-) 기간 설정
# -----------------------------------
# sub_df는 test data
# 코로나 최초발생일 = TEST_FIRST
TEST_FIRST = sub_df["Date"].min()
# train data 코로나 가장 마지막 발생 날 - 최초발생일
# ex. 최초발생일 1일 마지막발생날 5일 4, 5
TEST_DAYS = (df["Date"].max() - TEST_FIRST).days + 1
# 최초발생일보다 미만의 날짜 데이터 복사 -> dev_df
# 최초발생일 이후의 날짜 데이터 복사 ->test_df
dev_df, test_df = df[df["Date"] < TEST_FIRST].copy(), df[df["Date"] >= TEST_FIRST].copy()
dev_df.shape, test_df.shape

((19278, 8), (3978, 8))

In [14]:
dev_df

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,prev_ConfirmedCases,prev_Fatalities
1,2,none,Afghanistan,2020-01-23,0.000000,0.000000,0.000000,0.000000
2,3,none,Afghanistan,2020-01-24,0.000000,0.000000,0.000000,0.000000
3,4,none,Afghanistan,2020-01-25,0.000000,0.000000,0.000000,0.000000
4,5,none,Afghanistan,2020-01-26,0.000000,0.000000,0.000000,0.000000
5,6,none,Afghanistan,2020-01-27,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...
23544,32695,none,Zimbabwe,2020-03-21,1.386294,0.000000,0.693147,0.000000
23545,32696,none,Zimbabwe,2020-03-22,1.386294,0.000000,1.386294,0.000000
23546,32697,none,Zimbabwe,2020-03-23,1.386294,0.693147,1.386294,0.000000
23547,32698,none,Zimbabwe,2020-03-24,1.386294,0.693147,1.386294,0.693147


In [17]:
test_df

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,prev_ConfirmedCases,prev_Fatalities
64,65,none,Afghanistan,2020-03-26,4.553877,1.609438,4.442651,1.098612
65,66,none,Afghanistan,2020-03-27,4.709530,1.609438,4.553877,1.609438
66,67,none,Afghanistan,2020-03-28,4.709530,1.609438,4.709530,1.609438
67,68,none,Afghanistan,2020-03-29,4.795791,1.609438,4.709530,1.609438
68,69,none,Afghanistan,2020-03-30,5.141664,1.609438,4.795791,1.609438
...,...,...,...,...,...,...,...,...
23557,32708,none,Zimbabwe,2020-04-03,2.302585,0.693147,2.302585,0.693147
23558,32709,none,Zimbabwe,2020-04-04,2.302585,0.693147,2.302585,0.693147
23559,32710,none,Zimbabwe,2020-04-05,2.302585,0.693147,2.302585,0.693147
23560,32711,none,Zimbabwe,2020-04-06,2.397895,0.693147,2.302585,0.693147


In [15]:
sub_df["Date"].min()

Timestamp('2020-03-26 00:00:00')

# Regression 데이터예측모델

## Linear Regression 선형 회귀모델
- 회귀 모델 : 입력 변수 X에 대해서 연속형 출력함수 Y 예측
- 잔차 : 예측지와 실측지의 차이
- 데이터 세트의 목표와 선형 근사로 예측 된 목표 사이의 잔차 제곱합을 최소화
- 최소제곱법은 잔차의 제곱의 합이 최소가 되도록 하는 직선을 회귀선으로 한다는 것을 의미

![LinearRegression](https://t1.daumcdn.net/cfile/tistory/25487A4558D07B542B)

## MSE
### mean_squared_error 


평균제곱오차(MSE)는 
Regression 과 같은 실수 기반의 결과에 대한 오차를 판별하는 방식이다.

예시로, 실제 값이 100,000 원인 주식 가격을 내가 만든 모델이 95,000원 이라고 판별한다면, 이 모델이 얼마나 잘 판단한 것인지 애매하다.

따라서 실제 값과 예측값의 차이를 기준으로 오차를 판단하는 방식이다.

**MSE = (실제값 - 예측값)^2 / 크기**

#### 과적합 & 분산 & 편파성
![overfitting](https://t1.daumcdn.net/cfile/tistory/2358C33E5735DE8531)

In [17]:
from sklearn.linear_model import LinearRegression  # 선형 회귀 모델
from sklearn.metrics import mean_squared_error     # MSE (평균 제곱 오차)
from sklearn.preprocessing import PolynomialFeatures
# PolynomialFeatures : 입력 값 x를 다항식 변환
from sklearn.pipeline import Pipeline
# Pipeline : 전처리용 객체와 분류모형을 합치도록

# PolynomialFeatures : 다항변환
# degree 차수
# include_bias : 상수항 생성여부
# Pipeline -> array, shape (n_output_features, n_input_features)

model = Pipeline([('poly', PolynomialFeatures(degree=2, include_bias=False)),
                  ('linear', LinearRegression())])

# TARGETS = ["ConfirmedCases", "Fatalities"]
# features = ["prev_ConfirmedCases", "prev_Fatalities"]
features = ["prev_{}".format(col) for col in TARGETS]

# fit(x, y) Compute number of output features.
# dev_df = 최초발생일 이전의 데이터
# 모델 학습
model.fit(dev_df[features], dev_df[TARGETS])
# MSE 평균제곱오차
# (실제확진자수 - 예측확진자수)^2
# (실제 사망자수 - 예측 사망자수) ^2
[mean_squared_error(dev_df[TARGETS[i]], model.predict(dev_df[features])[:, i]) for i in range(len(TARGETS))]

[0.04280332071852165, 0.010099021812703432]

## RMSE (Root Mean Squared Error)
- 예측값과 실제값을 뺀 후 제곱시킨 값들을 다 더하고, n으로 나눈다
- 그리고 루트를 씌운다
- 평균 제곱근 편차 or표준편차와 식이 동일
- 모델이 잘 트레이닝 되었는지 RMSE를 기준으로 판단
![RMSE](https://www.kharn.kr/data/photos/20170832/art_1502155948.png)

In [18]:
# RMSE 함수
def rmse(y_true, y_pred):
    return np.sqrt(mean_squared_error(y_true, y_pred))

#TARGETS = [확진자수, 사망자수]
# 실제값과 예측값의 표준편차를 error에 누적
# error = 확진자수 표준편차 + 사망자수 표준편차
# 모델을 평가
def evaluate(df):
    error = 0
    for col in TARGETS:
        error += rmse(df[col].values, df["pred_{}".format(col)].values)
    return np.round(error/len(TARGETS), 5)
#np.round 소수점 5번째 자리까지 반올림

# 예측 함수
def predict(test_df, first_day, num_days, val=False):
    # np.clip(배열, 최소값 기준, 최대값 기준) 
    # 최소값과 최대값 조건으로 값을 기준으로 해서, 이 범위 기준을 벗어나는 값에 대해서는 
    # 일괄적으로 최소값, 최대값으로 대치해줄 때 매우 편리
    # 사망자-확진자수 (최초발생일) 행 배열 최소None - 16
    # y_pred : 분류 모형이 예측한 값
    y_pred = np.clip(model.predict(test_df.loc[test_df["Date"] == first_day][features]), None, 16)
    # for     in enumerate(); 열거형 내장함수
    #TARGETS = [확진자수, 사망자수]
    for i, col in enumerate(TARGETS):
        # pred_확진자 = 0, pred_사망자 = 0;=
        test_df["pred_{}".format(col)] = 0
        test_df.loc[test_df["Date"] == first_day, "pred_{}".format(col)] = y_pred[:, i]
        # [ : , i] 처음부터 끝까지 i번씩 건너띠면서
        
    if val:
        print(first_day, evaluate(test_df[test_df["Date"] == first_day]))
    # rmse 평균값을 출력
    # first_day 그 이후에
    for d in range(1, num_days):
        y_pred = np.clip(model.predict(y_pred), None, 16)
        date = first_day + timedelta(days=d)
        # timedelta -> ms 마이크로초
        for i, col in enumerate(TARGETS):
            test_df.loc[test_df["Date"] == date, "pred_{}".format(col)] = y_pred[:, i]

        if val:     # val이 true라면
            print(date, evaluate(test_df[test_df["Date"] == date]))
            # rmse 에러누적 평균
    return test_df

# predict 함수
test_df = predict(test_df, TEST_FIRST, TEST_DAYS, val=True)
evaluate(test_df)

2020-03-26 00:00:00 0.25798
2020-03-27 00:00:00 0.32955
2020-03-28 00:00:00 0.39398
2020-03-29 00:00:00 0.45843
2020-03-30 00:00:00 0.51629
2020-03-31 00:00:00 0.58445
2020-04-01 00:00:00 0.64146
2020-04-02 00:00:00 0.6987
2020-04-03 00:00:00 0.74463
2020-04-04 00:00:00 0.79215
2020-04-05 00:00:00 0.83994
2020-04-06 00:00:00 0.88312
2020-04-07 00:00:00 0.92971


0.65549

In [19]:
# np.expm1 : 배열의 모든 요소를 지수승 계산 합니다. (exp(x) - 1)
# 더 높은 정확도

for col in TARGETS:
    test_df[col] = np.expm1(test_df[col])
    # pred_ {col} 역시 np.expm1의 함수를 통해 지수승 계산
    test_df["pred_{}".format(col)] = np.expm1(test_df["pred_{}".format(col)])

In [22]:
# sub_df test데이터
SUB_FIRST = sub_df["Date"].min()   # 테스트 시작 날
SUB_DAYS = (sub_df["Date"].max() - sub_df["Date"].min()).days + 1   # 테스트기간
# test데이터 시작과 끝 43일

sub_df = dev_df.append(sub_df, sort=False) # 최초발생일 이전의 자료에 sub_df (+)
# 다시 합쳐서 원본으로 = sub_df
# prev_확진자, prev_사망자 추가
# loc_group = ["Province_State", "Country_Region"]
# shift 인덱스 이동
for col in TARGETS:
    sub_df["prev_{}".format(col)] = sub_df.groupby(loc_group)[col].shift()
# 최초 발생일 보다 크거나 같은 경우의 데이터 복사 저장    
sub_df = sub_df[sub_df["Date"] >= SUB_FIRST].copy()

# sub_df에 ForecastId 열 추가( dtype 변환 -> np.int16)
sub_df["ForecastId"] = sub_df["ForecastId"].astype(np.int16)
# predict함수 호출
sub_df = predict(sub_df, SUB_FIRST, SUB_DAYS)
# np.expm1 : 배열의 모든 요소를 지수승 계산 합니다. (exp(x) - 1)
# 더 높은 정확도
for col in TARGETS:
    sub_df[col] = np.expm1(sub_df["pred_{}".format(col)])
    
sub_df.head()

,Id,Province_State,Country_Region,Date,ConfirmedCases,Fatalities,prev_ConfirmedCases,prev_Fatalities,ForecastId,pred_ConfirmedCases,pred_Fatalities
0,NaN,none,Afghanistan,2020-03-26,99.477559,2.215166,4.442651,1.098612,1,4.609934,1.167879
1,NaN,none,Afghanistan,2020-03-27,117.369800,2.457977,99.477559,2.215166,2,4.773814,1.240684
2,NaN,none,Afghanistan,2020-03-28,137.924500,2.732247,117.369800,2.457977,3,4.933931,1.317011
3,NaN,none,Afghanistan,2020-03-29,161.391187,3.042347,137.924500,2.732247,4,5.090008,1.396825
4,NaN,none,Afghanistan,2020-03-30,188.019210,3.393287,161.391187,3.042347,5,5.241849,1.480078


In [21]:
(sub_df["Date"].max() - sub_df["Date"].min()).days+1

43

In [33]:
# DataFrame.to_csv() dataframe을 csv 파일로 내보내는 것
# index = False 인덱스 쓰지 않기\
# columns = 특정 칼럼이름을 적어주기
sub_df.to_csv("submission.csv", index=False, columns=["ForecastId"] + TARGETS)